<a href="https://colab.research.google.com/github/RohanOpenSource/Deep-Learning-And-Beyond/blob/main/BertClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import pandas as pd
from google.colab import files
! unzip train.zip

Archive:  train.zip
replace train/20230707_12_SN17_T1/overhead_view/20230707_12_SN17_T1_caption.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os,json

scraped = []

for root, dirs, files in os.walk(r'train'):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.json'):
            # print whole path of files
            scraped.append(os.path.join(root, file))
scraped

['train/20231006_23_CN2_T1/vehicle_view/20231006_23_CN2_T1_caption.json',
 'train/20231006_23_CN2_T1/overhead_view/20231006_23_CN2_T1_caption.json',
 'train/20230929_1_CN6_T3/vehicle_view/20230929_1_CN6_T3_caption.json',
 'train/20230929_1_CN6_T3/overhead_view/20230929_1_CN6_T3_caption.json',
 'train/20230922_47_SN37_T1/vehicle_view/20230922_47_SN37_T1_caption.json',
 'train/20230922_47_SN37_T1/overhead_view/20230922_47_SN37_T1_caption.json',
 'train/20230728_40_SN31_T1/overhead_view/20230728_40_SN31_T1_caption.json',
 'train/20230929_7_CN34_T1/vehicle_view/20230929_7_CN34_T1_caption.json',
 'train/20230929_7_CN34_T1/overhead_view/20230929_7_CN34_T1_caption.json',
 'train/20230929_61_CN35_T1/vehicle_view/20230929_61_CN35_T1_caption.json',
 'train/20230929_61_CN35_T1/overhead_view/20230929_61_CN35_T1_caption.json',
 'train/20230728_15_SY18_T2/vehicle_view/20230728_15_SY18_T2_caption.json',
 'train/20230728_15_SY18_T2/overhead_view/20230728_15_SY18_T2_caption.json',
 'train/20230728_7_SN

In [46]:
text_appended = ""
for f in scraped:
    with open(f, 'r') as infile:
        curr = (json.load(infile))
        series_curr = curr['event_phase']
        for line in series_curr:
            text_appended += (line['caption_pedestrian'])
            text_appended += (line['caption_vehicle'])
text_appended

'The pedestrian, a male in his 30s, stood directly in front of the vehicle, his body facing the opposite direction. He was closely watching the vehicle, noticing its presence. He had a clear line of sight, and despite the dim brightness, he was able to see the vehicle clearly. The pedestrian\'s visual status was fully operational. He was slowly moving forward, taking small steps. He was dressed in a black T-shirt and black slacks, and stood at a height of 160 cm. The road surface was dry and level, with light traffic on the two-way residential road. There were no sidewalks on either side, and he observed that there were no street lights. The overall environment was calm and clear, indicating favorable conditions for the pedestrian to navigate the road.The vehicle is positioned in front of the pedestrian, at a close distance. The pedestrian is visible within the vehicle\'s field of view. The vehicle performed an emergency braking maneuver to avoid a collision with the pedestrian. The ve

In [49]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
final_data = sent_tokenize(text_appended)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 4.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [54]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(final_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [57]:
clusters = util.community_detection(sentence_embeddings, min_community_size= 500, threshold=0.75)

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", final_data[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", final_data[sentence_id])



Cluster 1, #3926 Elements 
	 The road is level asphalt with two-way traffic, and there is a light traffic volume.
	 The road is level asphalt with two-way traffic, and there is a light traffic volume.
	 The road, made of asphalt, is level and has two-way traffic with a light volume.
	 ...
	 The road has a light traffic volume and is illuminated by street lights.
	 The road is a residential road with two-way traffic and a light volume of traffic.
	 The road is a residential road with two-way traffic and a light volume of traffic.

Cluster 2, #1855 Elements 
	 The environment condition around the vehicle consists of a male pedestrian in his 30s with a height of 160 cm.
	 The environment conditions surrounding the vehicle include a male pedestrian in his 30s with a height of 160 cm.
	 The environment conditions surrounding the vehicle include a male pedestrian in his 30s with a height of 160 cm.
	 ...
	 These details comprise the event, providing a clear description of the vehicle, its s

In [61]:
with open('Clusters.txt', 'w') as writefile:
    for i, cluster in enumerate(clusters):
        writefile.write("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
        for sentence_id in cluster:
            writefile.write("\n" + final_data[sentence_id])
